In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os as os

In [ ]:
os.chdir("D:/Data/")  # Change to your working directory
coeff = pd.read_csv("Coefficients/coefs_Scarletfever.csv", index_col=0)  
cov_matrix = pd.read_csv("Coefficients/cov_matrix_Scarletfever.csv", index_col=0) 
nlag = 6   

panel = pd.read_csv("panel.csv", encoding='GBK', index_col=0)
tele_e_mean = panel['t_p_corr_running_e'].mean() 
tele_c_mean = panel['t_p_corr_running_c'].mean()
t_critical=  1.96 # 95% confidence level

In [7]:
def calculate_e_effect(coeff, cov_matrix):  
    contrast = np.zeros(len(coeff))
  
    for j in range(1, nlag+1):
        e_lag = f'e_lag{j}'
        if e_lag in coeff.index:
            contrast[coeff.index.get_loc(e_lag)] += 1
        
    
        for interaction_format in [f'e_lag{j}:t_p_corr_running_e', f't_p_corr_running_e:e_lag{j}']:
            if interaction_format in coeff.index:
                contrast[coeff.index.get_loc(interaction_format)] = tele_e_mean
                break
    beta = coeff.values.flatten()
    beta= np.nan_to_num(beta, nan=0.0)
    estimate = np.dot(contrast, beta)
    variance = contrast.T @ cov_matrix @ contrast
    se = np.sqrt(variance)

    return estimate, se

In [8]:
def calculate_c_effect(coeff, cov_matrix):  
    contrast = np.zeros(len(coeff))
    
    for j in range(1, nlag+1):
        c_lag = f'c_lag{j}'
        if c_lag in coeff.index:
            contrast[coeff.index.get_loc(c_lag)] += 1
        
        for interaction_format in [f'c_lag{j}:t_p_corr_running_c', f't_p_corr_running_c:c_lag{j}']:
            if interaction_format in coeff.index:
                contrast[coeff.index.get_loc(interaction_format)] = tele_c_mean
                break
    beta = coeff.values.flatten()
    beta = np.nan_to_num(beta, nan=0.0)
    estimate = np.dot(contrast, beta) 
    variance = contrast.T @ cov_matrix @ contrast
    se = np.sqrt(variance)

    return estimate, se

In [9]:
tele_values = np.linspace(-2, 2, 50) 
cov_matrix = np.nan_to_num(cov_matrix, nan=0.0)

results = []
est_e, se_e = calculate_e_effect(coeff, cov_matrix)
est_c, se_c = calculate_c_effect(coeff, cov_matrix)

results.append({
        'estimate_e': est_e,
        'lower': est_e - t_critical*se_e,  
        'upper': est_e + t_critical*se_e,
        'estimate_c': est_c*(-1),
        'lower_c': est_c*(-1) - t_critical*se_c,
        'upper_c': est_c*(-1) + t_critical*se_c
    })


In [10]:
print(results)

[{'estimate_e': -0.13280941706662241, 'lower': -0.28190733493821785, 'upper': 0.01628850080497299, 'estimate_c': -0.2907522637721963, 'lower_c': -0.360055937475845, 'upper_c': -0.2214485900685476}]
